In [124]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
import numpy as np
import datetime
from pathlib import Path
import os
import math
import yfinance as yf
import tkinter as tk
from tkinter import ttk
import warnings
warnings.filterwarnings("ignore")

## Getting request

In [4]:
def get_request(url):
    headers = {'User-Agent': 
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
    page = requests.get(url, headers = headers)
    
    if page.status_code == 200:
        pass
    else:
        print(f'Request failed with status code: {response.status_code}')
        
    return page


## Getting  Statement by year

In [5]:
def get_income_statement(ticker):
    page_income = get_request('https://marketcap.ru/en/stocks/' + ticker + '/financial-statements/income-statement')
    soup_income = BeautifulSoup(page_income.text, 'lxml')
    table1_income = soup_income.find('table', {'class':'table table-striped table-detail-stocks table-responsive table-responsive-sm table-responsive-md table-responsive-lg table-responsive-xl'})
    headers_income = []
    for i in table1_income.find_all('th'):
        if i.text == '':
            continue
        else:
            headers_income.append(i.text)
        
    indexes_income = []

    for i in table1_income.find_all('span'):
        indexes_income.append(i.text)
        
    mydata_income = pd.DataFrame(columns = headers_income, index=indexes_income)
    counter = 0
    for j in table1_income.find_all('tr')[1:]:
        row_data = j.find_all('td', text = re.compile(r'^-?\d+(\.\d+)? ?[TBMK]?$|^-$'))
        row = [i.text for i in row_data]
        if len(row) != len(mydata_income.iloc[counter]):
            while len(row) != len(mydata_income.iloc[counter]):
                row.append(np.NaN)
            mydata_income.iloc[counter] = row
        else:
            mydata_income.iloc[counter] = row
        
        counter += 1
        
    return mydata_income


def get_balance_statement(ticker):
    page_balance = get_request('https://marketcap.ru/en/stocks/' + ticker + '/financial-statements/balance-sheet')
    soup_balance = BeautifulSoup(page_balance.text, 'lxml')
    table1_balance = soup_balance.find('table', {'class':'table col-md-12 table-striped table-detail-stocks table-responsive table-responsive-sm table-responsive-md table-responsive-lg table-responsive-xl'})
    headers_balance = []
    for i in table1_balance.find_all('th'):
        if i.text == '':
            continue
        else:
            headers_balance.append(i.text)
        
    indexes_balance = []

    for i in table1_balance.find_all('span'):
        indexes_balance.append(i.text)
        
    mydata_balance = pd.DataFrame(columns = headers_balance, index=indexes_balance)
    
    counter = 0
    for j in table1_balance.find_all('tr')[1:]:
        row_data = j.find_all('td', text = re.compile(r'^-?\d+(\.\d+)? ?[TBMK]?$|^-$'))
        row = [i.text for i in row_data]
        if len(row) != len(mydata_balance.iloc[counter]):
            while len(row) != len(mydata_balance.iloc[counter]):
                row.append(np.NaN)
            mydata_balance.iloc[counter] = row
        else:
            mydata_balance.iloc[counter] = row
        counter +=1
        
    return mydata_balance


def get_cashflow_statement(ticker):
    page_cash = get_request('https://marketcap.ru/en/stocks/' + ticker + '/financial-statements/cash-flow')
    soup_cash = BeautifulSoup(page_cash.text, 'lxml')
    table1_cash = soup_cash.find('table', {'class':'table table-striped table-striped-columns table-detail-stocks table-responsive table-responsive-sm table-responsive-md table-responsive-lg table-responsive-xl'})
    
    headers_cash = []
    for i in table1_cash.find_all('th'):
        if i.text == '':
            continue
        else:
            headers_cash.append(i.text)
        
    indexes_cash = []

    for i in table1_cash.find_all('span'):
        indexes_cash.append(i.text)
        
    mydata_cash = pd.DataFrame(columns = headers_cash, index=indexes_cash)
    
    counter = 0
    
    for j in table1_cash.find_all('tr')[1:]:
        row_data = j.find_all('td', text = re.compile(r'^-?\d+(\.\d+)? ?[TBMK]?$|^-$'))
        row = [i.text for i in row_data]
    
        if len(row) != len(mydata_cash.iloc[counter]):
            while len(row) != len(mydata_cash.iloc[counter]):
                row.append(np.NaN)
            mydata_cash.iloc[counter] = row
        else:
            mydata_cash.iloc[counter] = row
        counter +=1 
    
    return mydata_cash


def get_financial_ratios(ticker):
    
    page_ratios = get_request('https://marketcap.ru/en/stocks/' + ticker + '/financial-ratios')
    soup_ratios = BeautifulSoup(page_ratios.text, 'lxml')
    table1_ratios = soup_ratios.find('table', {'class':'table col-md-12 table-striped table-detail-stocks table-responsive table-responsive-sm table-responsive-md table-responsive-lg table-responsive-xl'})
    
    headers_ratios = []
    for i in table1_ratios.find_all('th'):
        if i.text == '':
            continue
        else:
            headers_ratios.append(i.text)
        
    indexes_ratios = []

    for i in table1_ratios.find_all('span'):
        indexes_ratios.append(i.text)
        
    mydata_ratios = pd.DataFrame(columns = headers_ratios, index=indexes_ratios)
    
    counter = 0
    for j in table1_ratios.find_all('tr')[1:]:
        row_data = j.find_all('td', text = re.compile(r'^-?\d+(\.\d+)? ?[TBMK]?$|^-$'))
        row = [i.text for i in row_data]

        if len(row) != len(mydata_ratios.iloc[counter]):
            while len(row) != len(mydata_ratios.iloc[counter]):
                row.append(np.NaN)
            mydata_ratios.iloc[counter] = row
        else:
            mydata_ratios.iloc[counter] = row

        counter +=1  
    
    return mydata_ratios

## Getting Statements quarterly


In [6]:
def get_income_statement_quarterly(ticker):
    quarterly_income = get_request('https://marketcap.ru/en/stocks/' + ticker + '/financial-statements/income-statement')
    soup_income_quarterly = BeautifulSoup(quarterly_income.text, 'lxml')
    table1_income_quarterly = soup_income_quarterly.find_all('table', 
                                   {'class':'table col-md-12 table-striped table-detail-stocks table-responsive table-responsive-sm table-responsive-md table-responsive-lg table-responsive-xl'}
                                  )[-1]
    headers_income_quarterly = []
    for i in table1_income_quarterly.find_all('th'):
        if i.text == '':
            continue
        elif str(datetime.datetime.now().year - 1) in i.text:
            headers_income_quarterly.append(i.text)
        
    indexes_income_quarterly = []

    for i in table1_income_quarterly.find_all('span'):
        indexes_income_quarterly.append(i.text)
    
    mydata_income_quarterly = pd.DataFrame(columns = headers_income_quarterly, index=indexes_income_quarterly)
    counter = 0
    for j in table1_income_quarterly.find_all('tr')[1:]:
        row_data = j.find_all('td', text = re.compile(r'^-?\d+(\.\d+)? ?[TBMK]?$|^-$'))
        row = [i.text for i in row_data][:len(mydata_income_quarterly.columns)]
        if len(row) != len(mydata_income_quarterly.iloc[counter]):
            while len(row) != len(mydata_income_quarterly.iloc[counter]):
                row.append(np.NaN)
            mydata_income_quarterly.iloc[counter] = row
        else:
            mydata_income_quarterly.iloc[counter] = row
        counter +=1 
    
    return mydata_income_quarterly




def get_balance_statement_quarterly(ticker):
    quarterly_balance = get_request('https://marketcap.ru/en/stocks/' + ticker + '/financial-statements/balance-sheet')
    soup_balance_quarterly = BeautifulSoup(quarterly_balance.text, 'lxml')
    table1_balance_quarterly = soup_balance_quarterly.find_all('table', 
                                       {'class':'table col-md-12 table-striped table-detail-stocks table-responsive table-responsive-sm table-responsive-md table-responsive-lg table-responsive-xl'}
                                      )[-1]
    headers_balance_quarterly = []
    for i in table1_balance_quarterly.find_all('th'):
        if i.text == '':
            continue
        elif str(datetime.datetime.now().year - 1) in i.text:
            headers_balance_quarterly.append(i.text)
        
    indexes_balance_quarterly = []

    for i in table1_balance_quarterly.find_all('span'):
        indexes_balance_quarterly.append(i.text)
        
    mydata_balance_quarterly = pd.DataFrame(columns = headers_balance_quarterly, index=indexes_balance_quarterly)
    
    counter = 0
    for j in table1_balance_quarterly.find_all('tr')[1:]:
        row_data = j.find_all('td', text = re.compile(r'^-?\d+(\.\d+)? ?[TBMK]?$|^-$'))
        row = [i.text for i in row_data][:len(mydata_balance_quarterly.columns)]
        if len(row) != len(mydata_balance_quarterly.iloc[counter]):
            while len(row) != len(mydata_balance_quarterly.iloc[counter]):
                row.append(np.NaN)
            mydata_balance_quarterly.iloc[counter] = row
        else:
            mydata_balance_quarterly.iloc[counter] = row
        counter +=1 
    
    
    return mydata_balance_quarterly



def get_cashflow_statement_quarterly(ticker):
    quarterly_cash = get_request('https://marketcap.ru/en/stocks/' + ticker + '/financial-statements/cash-flow')
    soup_cash_quarterly = BeautifulSoup(quarterly_cash.text, 'lxml')
    table1_cash_quarterly = soup_cash_quarterly.find_all('table', 
                                       {'class':'table col-md-12 table-striped table-detail-stocks table-responsive table-responsive-sm table-responsive-md table-responsive-lg table-responsive-xl'}
                                      )[-1]
    headers_cash_quarterly = []
    for i in table1_cash_quarterly.find_all('th'):
        if i.text == '':
            continue
        elif str(datetime.datetime.now().year - 1) in i.text:
            headers_cash_quarterly.append(i.text)

    indexes_cash_quarterly = []

    for i in table1_cash_quarterly.find_all('span'):
        indexes_cash_quarterly.append(i.text)
        
    mydata_cash_quarterly = pd.DataFrame(columns = headers_cash_quarterly, index=indexes_cash_quarterly)
    
    counter = 0
    for j in table1_cash_quarterly.find_all('tr')[1:]:
        row_data = j.find_all('td', text = re.compile(r'^-?\d+(\.\d+)? ?[TBMK]?$|^-$'))
        row = [i.text for i in row_data][:len(mydata_cash_quarterly.columns)]
        if len(row) != len(mydata_cash_quarterly.iloc[counter]):
            while len(row) != len(mydata_cash_quarterly.iloc[counter]):
                row.append(np.NaN)
            mydata_cash_quarterly.iloc[counter] = row
        else:
            mydata_cash_quarterly.iloc[counter] = row
        counter +=1 
    
    return mydata_cash_quarterly



def get_financial_ratios_quarterly(ticker):
    quarterly_ratios = get_request('https://marketcap.ru/en/stocks/' + ticker + '/financial-ratios')
    soup_ratios_quarterly = BeautifulSoup(quarterly_ratios.text, 'lxml')
    table1_ratios_quarterly = soup_ratios_quarterly.find_all('table', 
                                       {'class':'table col-md-12 table-striped table-detail-stocks table-responsive table-responsive-sm table-responsive-md table-responsive-lg table-responsive-xl'}
                                      )[-1]
    
    headers_ratios_quarterly = []
    for i in table1_ratios_quarterly.find_all('th'):
        if i.text == '':
            continue
        elif str(datetime.datetime.now().year - 1) in i.text:
            headers_ratios_quarterly.append(i.text)

    indexes_ratios_quarterly = []

    for i in table1_ratios_quarterly.find_all('span'):
        indexes_ratios_quarterly.append(i.text)
        
    mydata_ratios_quarterly = pd.DataFrame(columns = headers_ratios_quarterly, index=indexes_ratios_quarterly)
    
    counter = 0
    for j in table1_ratios_quarterly.find_all('tr')[1:]:
        row_data = j.find_all('td', text = re.compile(r'^-?\d+(\.\d+)? ?[TBMK]?$|^-$'))
        row = [i.text for i in row_data][:len(mydata_ratios_quarterly.columns)]
        if len(row) != len(mydata_ratios_quarterly.iloc[counter]):
            while len(row) != len(mydata_ratios_quarterly.iloc[counter]):
                row.append(np.NaN)
            mydata_ratios_quarterly.iloc[counter] = row
        else:
            mydata_ratios_quarterly.iloc[counter] = row
        counter +=1 
    
    return mydata_ratios_quarterly

In [7]:
def convert_to_number(value):
    if value == '-':
        return np.nan
    elif isinstance(value, str):
        if 'T' in value:
            return float(value.replace('T', '')) * 1000
        elif 'B' in value:
            return float(value.replace('B', ''))
        elif 'M' in value:
            return float(value.replace('M', '')) * 0.001
        elif 'K' in value:
            return float(value.replace('K', '')) * 0.000001
        else:
            return value
    else:
        return value

## Get Final Reports

In [8]:
def final_income_statement_report(ticker):
    
    final_report = pd.DataFrame()
    
    if str(datetime.datetime.now().year - 1) not in get_income_statement(ticker).columns:
        final_report = pd.concat([get_income_statement_quarterly(ticker), get_income_statement(ticker)], axis=1)
        final_report = final_report.applymap(convert_to_number)
        return final_report
    else:
        return get_income_statement(ticker).applymap(convert_to_number)
        

def final_balance_statement_report(ticker):
    
    final_report = pd.DataFrame()
    
    if str(datetime.datetime.now().year - 1) not in get_balance_statement(ticker).columns:
        final_report = pd.concat([get_balance_statement_quarterly(ticker), get_balance_statement(ticker)], axis=1)
        final_report = final_report.applymap(convert_to_number)
        return final_report
    else:
        return get_balance_statement(ticker).applymap(convert_to_number)

def final_cashflow_statement_report(ticker):
    
    final_report = pd.DataFrame()
    
    if str(datetime.datetime.now().year - 1) not in get_cashflow_statement(ticker).columns:
        final_report = pd.concat([get_cashflow_statement_quarterly(ticker), get_cashflow_statement(ticker)], axis=1)
        final_report = final_report.applymap(convert_to_number)
        return final_report
    else:
        return get_cashflow_statement(ticker).applymap(convert_to_number)

def final_ratios_statement_report(ticker):
    
    final_report = pd.DataFrame()
    
    if str(datetime.datetime.now().year - 1) not in get_financial_ratios(ticker).columns:
        final_report = pd.concat([get_financial_ratios_quarterly(ticker), get_financial_ratios(ticker)], axis=1)
        final_report = final_report.applymap(convert_to_number)
        return final_report
    else:
        return get_financial_ratios(ticker).applymap(convert_to_number)

## Saving all reports

In [9]:
def get_all_statements(ticker):
    t = ticker
    data_path = Path("FinancialData/")
    company_path = data_path / t
    
    if company_path.is_dir():
        print(f"{company_path} directory exists.")
    else:
        print(f"Did not find {company_path} directory, creating one...")
        company_path.mkdir(parents=True, exist_ok=True)
        
    
    final_income_statement_report(ticker).to_csv(company_path/f"{ticker + ' ' + 'income_statement'}.csv")
    final_cashflow_statement_report(ticker).to_csv(company_path/f"{ticker + ' ' + 'cashflow_statement'}.csv")
    final_balance_statement_report(ticker).to_csv(company_path/f"{ticker + ' ' + 'balance_statement'}.csv")
    final_ratios_statement_report(ticker).to_csv(company_path/f"{ticker + ' ' + 'financial_ratios'}.csv")

In [10]:
def get_dividends(ticker):
    company = yf.Ticker(ticker)
    query = company.dividends[company.dividends.index.year >= int(final_income_statement_report(ticker).T.index.min())]
    total_div_by_year = query.groupby(by = query.index.year).sum()
    total_div_by_year.index = total_div_by_year.index.astype(str)
    dividends_df = total_div_by_year.to_frame(name='Dividends')
    
    return dividends_df

In [11]:
def get_pe_ratio(company):
    pe = get_request('https://companiesmarketcap.com/' + company + '/pe-ratio/')
    page_pe = BeautifulSoup(pe.text, 'lxml')
    table1_pe = page_pe.find('table', {'class':'table'})
    headers_pe = []
    for i in table1_pe.find_all('th'):
        if i.text == '':
                continue
        else:
            headers_pe.append(i.text)

    pe_data = pd.DataFrame(columns = headers_pe, index=range(len(table1_pe.find_all('tr')[1:])))
    
    counter = 0

    for j in table1_pe.find_all('tr')[1:]:
        row_data = j.find_all('td', text = re.compile(r'[-+]?\$?\d+(\.\d+)?'))
        row = [i.text for i in row_data]

        if len(row) != len(pe_data.iloc[counter]):
            while len(row) != len(pe_data.iloc[counter]):
                row.append(np.NaN)
            pe_data.iloc[counter] = row
        else:
            pe_data.iloc[counter] = row

        counter += 1
        
    pe_data.drop(columns=['Change'], inplace = True)
    pe_data.set_index('Year', drop=True, inplace=True)
    pe_data.rename(index={'2023 (TTM)': '2023', '2024 (TTM)': '2024'}, inplace=True)
    pe_data.index = pe_data.index.astype(str)
    pe_data['P/E ratio'] = pe_data['P/E ratio'].str.replace('<', '')
    pe_data['P/E ratio'] = pe_data['P/E ratio'].astype(float)
    pe_data['P/E ratio'] = pe_data['P/E ratio'].apply(lambda x: 0 if x < 0 else x)

        
    return pe_data

In [12]:
def get_eps(company):
    eps = get_request('https://companiesmarketcap.com/' + company + '/eps/')
    page_eps = BeautifulSoup(eps.text, 'lxml')
    table1_eps = page_eps.find('table', {'class':'table'})
    headers_eps = []
    for i in table1_eps.find_all('th'):
        if i.text == '':
                continue
        else:
            headers_eps.append(i.text)

    eps_data = pd.DataFrame(columns = headers_eps, index=range(len(table1_eps.find_all('tr')[1:])))
    
    counter = 0

    for j in table1_eps.find_all('tr')[1:]:
        row_data = j.find_all('td', text = re.compile(r'[-+]?\$?\d+(\.\d+)?'))
        row = [i.text for i in row_data]

        if len(row) != len(eps_data.iloc[counter]):
            while len(row) != len(eps_data.iloc[counter]):
                row.append(np.NaN)
            eps_data.iloc[counter] = row
        else:
            eps_data.iloc[counter] = row

        counter += 1
        
    eps_data.drop(columns=['Change'], inplace = True)
    eps_data.set_index('Year', drop=True, inplace=True)
    eps_data.rename(index={'2023 (TTM)': '2023', '2024 (TTM)': '2024'}, inplace=True)
    eps_data.index = eps_data.index.astype(str)
    eps_data['EPS'] = eps_data['EPS'].str.replace('$', '')
    eps_data['EPS'] = eps_data['EPS'].astype(float)
        
    return eps_data
    

In [13]:
def get_revenue(company):
    revenue = get_request('https://companiesmarketcap.com/' + company + '/revenue/')
    page_revenue = BeautifulSoup(revenue.text, 'lxml')
    table1_revenue = page_revenue.find('table', {'class':'table'})
    headers_revenue = []
    for i in table1_revenue.find_all('th')[1:]:
        if i.text == '':
                continue
        else:
            headers_revenue.append(i.text)
            
    indexes_revenue = []

    for i in table1_revenue.find_all('span'):
        indexes_revenue.append(i.text)

    revenue_data = pd.DataFrame(columns = headers_revenue, index=indexes_revenue)

    
    counter = 0

    for j in table1_revenue.find_all('tr')[1:]:
        row_data = j.find_all('td', text = re.compile(r'[-+]?\$?\d+(\.\d+)?'))
        row = [i.text for i in row_data]

        if len(row) != len(revenue_data.iloc[counter]):
            while len(row) != len(revenue_data.iloc[counter]):
                row.append(np.NaN)
            revenue_data.iloc[counter] = row
        else:
            revenue_data.iloc[counter] = row

        counter += 1
        
    revenue_data.drop(columns=['Change'], inplace = True)
    revenue_data.rename(index={'2023 (TTM)': '2023', '2024 (TTM)': '2024'}, inplace=True)
    revenue_data.index = revenue_data.index.astype(str)
    revenue_data['Revenue'] = revenue_data['Revenue'].str.replace('$', '')
    revenue_data['Revenue'] = revenue_data['Revenue'].str.replace('B', '')
    revenue_data['Revenue'] = revenue_data['Revenue'].astype(float)

        
    return revenue_data

In [14]:
def main_statements(ticker, company):
    income = final_income_statement_report(ticker)
    balance = final_balance_statement_report(ticker)
    cashflow = final_cashflow_statement_report(ticker)
    ratios = final_ratios_statement_report(ticker)
    
    concatenated_tables = pd.concat([income, balance, cashflow, ratios], axis = 0)
    res = concatenated_tables.loc[['Net Income', 'Total Shareholders Equity', 'ROE', 'Long Term Debt',
                                   'P/E']]
    
    res = res.T
    eps = get_eps(company)
    div = get_dividends(ticker)
    
    pe_df = get_pe_ratio(company)
    revenue = get_revenue(company)
    
    
    res = res.join([eps, div, pe_df, revenue])
    res['P/E'] = res['P/E ratio'].combine_first(res['P/E'])
    res.drop(columns=['P/E ratio'], inplace=True)
    
    return res
    

In [130]:
def metrics(num_of_year, ticker, company):
    print(f'Отчет подготовлен по компании: {ticker}')
    print(f'Период: {num_of_year} лет')
    df = main_statements(ticker, company)
    
    columns_to_convert = [
    'Net Income', 'Total Shareholders Equity', 'ROE', 
    'Long Term Debt', 'EPS', 'Revenue', 'P/E']

    df[columns_to_convert] = df[columns_to_convert].astype(float)
    
    
    df['LTD/NPR'] = np.where(df['Net Income'] > 0, np.floor(df['Long Term Debt'] / df['Net Income'] * 100) / 100, np.nan)
    df['ROS'] = (df['Net Income'] / df['Revenue']) * 100
    
    average_annual_profit = round(((df['Net Income'][0] / abs(df['Net Income'][0:num_of_year][-1]))**(1/(num_of_year-1)) * 100) - 100, 2)
    average_annual_equity = round(((df['Total Shareholders Equity'][0] / df['Total Shareholders Equity'][0:num_of_year][-1])**(1/(num_of_year-1)) * 100) - 100, 2)
    average_roe = round(df['ROE'][0:num_of_year].mean(),2)
    average_LTD_NPR = round(df['LTD/NPR'][0:num_of_year].mean(),2)
    profit_growth = ((df['Net Income'][0] - df['Net Income'][0:num_of_year][-1]) / df['Net Income'][0:num_of_year][-1]) * 100
    profit_growth = round(profit_growth, 2) 
    efficiency_managers = (df['EPS'][0] - df['EPS'][0:num_of_year][-1]) / (df['EPS'][0:num_of_year].sum() - df['Dividends'][0:num_of_year].sum()) * 100
    efficiency_managers = round(efficiency_managers, 2)
    
    average_ros = round(df['ROS'][0:num_of_year].mean(),2)
    
    print(f'Среднегодовой темп прироста прибыли: {average_annual_profit} %')
    print(f'Среднегодовой темп прироста cобственного капитала: {average_annual_equity} %')
    print(f'Среднее значение ROE: {average_roe} %')
    print(f'Среднее значение отношения ДД/ЧП: {average_LTD_NPR}')
    print(f'Рост прибыли  за {num_of_year} лет: {profit_growth} %')
    print(f'Показатель эффективности менеджмента  за {num_of_year} лет: {efficiency_managers} %')
    print(f'Среднее значение ROS: {average_ros} %')
    
    loss_years_count = (df['Net Income'][:num_years] < 0).sum()

    loss_recovery = True
    if loss_years_count > 0:
        last_loss_year = df['Net Income'][:num_years][df['Net Income'][:num_years] < 0].index[-1]
        last_loss_value = df['Net Income'][last_loss_year]
        recovery_years = df['Net Income'][:last_loss_year].index[:-1]
        recovery_values = df['Net Income'][recovery_years]
        if not any(recovery_values >= last_loss_value) and (recovery_values < 0):
            loss_recovery = False
            
    get_all_statements(ticker)
    
    scores = {'A': 0, 'B': 0, 'C': 0, 'D': 0}
    
    conditions = {
    'A': [
        (average_annual_profit >= 15),
        (average_annual_equity > 10),
        (average_roe >= 15),
        (0<= average_LTD_NPR <= 5)
    ],
    'B': [
        (5 < average_annual_profit < 15),
        (5 < average_annual_equity <= 10),
        (average_roe <= 10),
        (5 < average_LTD_NPR <= 7),
        (loss_years_count <= 2) and loss_recovery
    ],
    'C': [
        (0 <= np.sum(df['Net Income']) <= 1),
        (3 <= average_annual_equity <= 5),
        (average_roe < 0),
        (average_LTD_NPR > 10)
    ],
    'D': [
        (np.sum(df['Net Income']) < 0),
        (average_LTD_NPR > 10),
        (average_roe < 0)
    ]
}

    for category, conds in conditions.items():
        scores[category] = sum(conds)


    category = max(scores, key=scores.get)
    
    print(f'Рекомендуется относить к категории {category}')
    
    t = ticker
    data_path = Path("FinancialData/")
    company_path = data_path / t
    
    if company_path.is_dir():
        print(f"{company_path} directory exists.")
    else:
        print(f"Did not find {company_path} directory, creating one...")
        company_path.mkdir(parents=True, exist_ok=True)
    
    df.to_csv(company_path/f"{ticker + ' ' + 'main_statements_for_analysis'}.csv")
    
    return df, scores, average_annual_profit, average_annual_equity, average_roe, average_LTD_NPR, profit_growth,efficiency_managers, average_ros


In [134]:
def run_metrics():
    ticker = ticker_entry.get()
    company = company_entry.get()
    num_of_year = int(years_entry.get())
    df, scores, average_annual_profit, average_annual_equity, average_roe, average_LTD_NPR, profit_growth,efficiency_managers, average_ros = metrics(num_of_year, ticker, company)
    result_text.delete("1.0", tk.END)
    result_text.insert(tk.END, f'Результаты для компании {ticker}:\n\n')
    result_text.insert(tk.END, df.to_string())
    result_text.insert(tk.END, f'\n\nКатегория: {max(scores, key=scores.get)}')
    result_text.insert(tk.END, f'\n\nСреднегодовой темп прироста прибыли: {average_annual_profit} %')
    result_text.insert(tk.END, f'\nСреднегодовой темп прироста cобственного капитала: {average_annual_equity} %')
    result_text.insert(tk.END, f'\nСреднее значение ROE: {average_roe} %')
    result_text.insert(tk.END, f'\nСреднее значение отношения ДД/ЧП: {average_LTD_NPR}')
    result_text.insert(tk.END, f'\nРост прибыли  за {num_of_year} лет: {profit_growth} %')
    result_text.insert(tk.END, f'\nПоказатель эффективности менеджмента  за {num_of_year} лет: {efficiency_managers} %')
    result_text.insert(tk.END, f'\nСреднее значение ROS: {average_ros} %')
    

    

root = tk.Tk()
root.title("Company Metrics")

ttk.Label(root, text="Ticker:").grid(column=0, row=0, padx=10, pady=5)
ticker_entry = ttk.Entry(root)
ticker_entry.grid(column=1, row=0, padx=10, pady=5)

ttk.Label(root, text="Company:").grid(column=0, row=1, padx=10, pady=5)
company_entry = ttk.Entry(root)
company_entry.grid(column=1, row=1, padx=10, pady=5)

ttk.Label(root, text="Number of Years:").grid(column=0, row=2, padx=10, pady=5)
years_entry = ttk.Entry(root)
years_entry.grid(column=1, row=2, padx=10, pady=5)

run_button = ttk.Button(root, text="Run Metrics", command=run_metrics)
run_button.grid(column=0, row=3, columnspan=2, pady=10)

result_text = tk.Text(root, width=180, height=40)
result_text.grid(column=0, row=4, columnspan=2, padx=10, pady=10)

root.mainloop()

Отчет подготовлен по компании: AMZN
Период: 5 лет
Среднегодовой темп прироста прибыли: 27.23 %
Среднегодовой темп прироста cобственного капитала: 34.3 %
Среднее значение ROE: 15.77 %
Среднее значение отношения ДД/ЧП: 1.71
Рост прибыли  за 5 лет: 162.07 %
Показатель эффективности менеджмента  за 5 лет: 9.53 %
Среднее значение ROS: 4.3 %
FinancialData\AMZN directory exists.
Рекомендуется относить к категории A
FinancialData\AMZN directory exists.
